In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install imutils

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import os
import random 
import cv2
import imutils
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
import keras
from keras.utils import np_utils
from keras.models import Model, Sequential
from keras import optimizers
from sklearn.preprocessing import LabelBinarizer
from keras import backend as K
from keras.layers import Dense, Activation, Flatten, Dense,MaxPooling2D, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

np.random.seed(123)

In [4]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs available:  1


In [13]:
train_dir = 'dataset/handwritten_character/Train'
val_dir = 'dataset/handwritten_character/Validation'

idg = ImageDataGenerator(rescale=1./255)

train_generator = idg.flow_from_directory(train_dir, color_mode = 'grayscale',
                                         target_size=(32, 32),
                                         batch_size = 32,
                                         class_mode ='categorical')

valid_generator = idg.flow_from_directory(val_dir, color_mode = 'grayscale',
                                         target_size=(32, 32),
                                         batch_size = 32,
                                         class_mode ='categorical')

train_images, train_labels = next(train_generator)
val_images, test_labels = next(valid_generator)

Found 834036 images belonging to 39 classes.
Found 22524 images belonging to 39 classes.


In [14]:
train_images.shape

(32, 32, 32, 1)

In [6]:
train_data = []
img_size = 32
non_chars = ["#", "$","&","@"]
for i in os.listdir(train_dir):
    if i in non_chars:
        continue
    count = 0
    sub_directory = os.path.join(train_dir, i)
    for j in os.listdir(sub_directory):
        count+=1
        if count > 4000:
            break
        img = cv2.imread(os.path.join(sub_directory,j), 0)
        img = cv2.resize(img,(img_size,img_size))
        train_data.append([img,i])

In [7]:
len(train_data)

140000

In [8]:
train_data[0:3]

[[array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 2, ..., 0, 0, 0],
         [1, 1, 0, ..., 0, 0, 0],
         [1, 0, 0, ..., 0, 0, 0]], dtype=uint8),
  '0'],
 [array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 1, 1, 0],
         [0, 0, 0, ..., 0, 0, 1],
         ...,
         [0, 2, 1, ..., 0, 0, 0],
         [0, 1, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
  '0'],
 [array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [1, 1, 0, ..., 0, 0, 0],
         [0, 0, 2, ..., 0, 0, 0],
         [1, 0, 0, ..., 0, 0, 0]], dtype=uint8),
  '0']]

In [10]:
val_data = []
img_size = 32
for i in os.listdir(val_dir):
    if i in non_chars:
        continue
    count = 0
    sub_directory = os.path.join(val_dir, i)
    for j in os.listdir(sub_directory):
        count+=1
        if count> 1000:
            break
        img = cv2.imread(os.path.join(sub_directory, j), 0)
        img = cv2.resize(img,(img_size, img_size))
        val_data.append([img,i])

In [11]:
len(val_data)

15209

In [12]:
np.random.seed(123)
random.shuffle(train_data)
random.shuffle(val_data)

In [13]:
train_X = []
train_y = []
for features, label in train_data:
    train_X.append(features)
    train_y.append(label)

In [14]:
val_X = []
val_y = []
for features, label in val_data:
    val_X.append(features)
    val_y.append(label)

In [15]:
LB = LabelBinarizer()
train_y = LB.fit_transform(train_y)
val_y = LB.fit_transform(val_y)

In [16]:
train_X = np.array(train_X)/255.0
train_X = train_X.reshape(-1,32,32,1)
train_y = np.array(train_y)

In [17]:
val_X = np.array(val_X)/255.0
val_X = val_X.reshape(-1,32,32,1)
val_y = np.array(val_y)

In [18]:
print("Training batch shape :", train_X.shape)
print("Validation batch shape :", val_X.shape)

Training batch shape : (140000, 32, 32, 1)
Validation batch shape : (15209, 32, 32, 1)


In [19]:
print("Training labels shape :", train_y.shape)
print("Validation labels shape :", val_y.shape)

Training labels shape : (140000, 35)
Validation labels shape : (15209, 35)


In [20]:
# restnet = ResNet50(include_top=False, weights='imagenet')

In [21]:
# output = restnet.layers[-1].output
# output= keras.layers.Flatten()(output)

In [22]:
# restnet = Model(restnet.input, output=output)

In [23]:
model_1 = Sequential()

model_1.add(Conv2D(32, (3, 3), padding = 'same', activation='relu', input_shape=(32, 32, 1)))
model_1.add(MaxPooling2D(pool_size=(2,2)))
model_1.add(Conv2D(64, (3, 3), activation='relu'))
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(Conv2D(128, (3, 3), activation='relu'))
model_1.add(MaxPooling2D(pool_size=(2,2)))
model_1.add(Dropout(.025))

model_1.add(Flatten())
model_1.add(Dense(128, activation='relu'))
model_1.add(Dropout(0.2))
model_1.add(Dense(35, activation='softmax'))

In [24]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 128)         0         
_________________________________________________________________
dropout (Dropout)            (None, 2, 2, 128)         0

In [25]:
model_1.compile(loss = 'categorical_crossentropy',
             optimizer='adam',
             metrics = ['accuracy'])

In [26]:
# np.random.seed(123)
# results_1 = model_1.fit(train_X, train_y, epochs=50, batch_size=32, validation_data=(val_X, val_y), verbose=1)

In [27]:
# try resnet model